# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
import random as rm
import delphi.evaluation_port as EN
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from delphi.cpp.DelphiPython import AnalysisGraph as AG, InitialBeta as IB, RNG
import time
import matplotlib.pyplot as plt
from delphi.jupyter_tools import display
import seaborn as sns
jt.print_commit_hash_message()

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)
R = RNG.rng()
R.set_seed(87)

Here we load the Causal Analysis Graph (CAG) from a corpus stored in a json file. This will give us a large CAG that we have dubbed the reference CAG. In this notebook, we will show how the reference CAG can be pruned and centered around a specific concept.

In [ ]:
start_time = time.time()
G = AG.from_json_file("../scripts/data/raw/wm_12_month_evaluation/jsonld-merged20190404.json",0.0,0.0)

Here we use the remove_node method to remove some unwanted concepts, specifically ones we feel are ambiguous or too abstract. 

In [ ]:
G.remove_node("UN/events/human/physical_insecurity")
G.remove_node("UN/events/nature_impact/climate_change_mitigation")
G.remove_node("UN/interventions/provision_of_goods_and_services")
G.remove_node("UN/entities/human/infrastructure/transportation/transportation_methods")
G.remove_node("UN/entities/human/government/government_actions/duty")
#G.remove_node("UN/events/human/conflict")

This method below will center our CAG around the "Human Migration" concept and remove all concepts that have more than a depth of 2 away from "Human Migration". The given True parameter indicates that we want to specifically look at causal flow from other concepts to "Human Migration" (i.e., specifically how do other concepts affect "Human Migration".

In [ ]:
H = G.get_subgraph_for_concept("UN/events/human/human_migration",4,True)

Next we attach indicators to the concepts where we first call map_concepts_to_indicators() to attach indicators to concepts according to a numerical score we call our grounding score. This score measures how much evidence we have that an indicator is relevant to the concept. 

If we are not satisfied with an attached indicator, we can replace it with one of our choosing by using replace_indicator(). 

In [ ]:
H.map_concepts_to_indicators()

H.replace_indicator("UN/events/human/human_migration","Net migration","New asylum seeking applicants", "UNHCR")


The above corpus that we load and use to create the CAG also may contain an adjective for each connected concept pair that indicates how one concept may affect the other concept in terms of increases or decreases. We use this information to construct prior probability distributions for concept pairs. 

In [ ]:
H.construct_beta_pdfs()

Here we can see a visual of the built CAG with the attached indicators.

In [ ]:
display(H)

Here we train the inference model for the Casual Analysis Graph. Below you can see that the CAG H calls the train_model method. 

Other important arguments are:
- start_year: The initial year to start training from.
- start_month: The initial month to start training from.
- end_year: The ending year for training.
- end_month: The ending month for training. 

The above arguments ensures that the model is trained with the appropriate data given a time range.

The 5th argument shown is the the sample resolution (default is 200) and the 6th argument is the number of samples to burn in the mcmc sampler before retaining samples. Finally the last argument sets the initial conditions for the mcmc sampler, IB.ZERO sets the betas initially to zero.  

The train_model function can also take in all the parameter arguments as parameterize allowing for the setting of country, state and units. 

In [ ]:
H.train_model(2011,7,2018,10,100,900,initial_beta=IB.ZERO,use_heuristic=True)

The next function generates predictions for a given time range. Like train_model this takes a set of arguments start_year, start_month, end_year, and end_month that specify the time range for predictions. The function returns a tuple with the first element containing a list of the prediction dates and the second element containing a nested data structure. The nested structure is a list containing a list for each sample (as set by res in train_model()). Each list within the outer list contains a dictionary for each time step in the prediction range (including the 0th time step). The keys of these dictionary objects are the node names as strings and the values are dictionary objects themselves. The keys of these inner dictionary objects are the indicator names as string and the values are the prediction values.

*Note: The predictions can be heavily reliant of the initial conditions, which are determined by the initial date of the prediction range (i.e, I suspect there is an initial condition bias). It still remains to be tested whether or not starting predictions from the initial training date or starting at the end of the training range yields more accuracte predictions. For example if training from January, 2015 to December, 2015 and we want to get predictions for January, 2016 to December, 2016, is it better to start predicting from January, 2015 or at the start of the dates we want (January, 2016). Initiating predictions at one time step before the prediction range is also a possibility for the most accurate predictions.

In [ ]:
preds = H.generate_prediction(2011,7,2012,12)
end_time = time.time()

total_time = end_time-start_time

This is the total computation time (in seconds) from loading and building the CAG to generating predictions. 

In [ ]:
total_time

Now that the predictions have been generated, a user can store or present the predictions however they choose. However the evaluation module comes with several convienant options for displaying output for a specific indicator. The first option to just return the raw predictions for a given indicator variable in a numpy array. This allows one to do there own plotting and manipulations for a given indicator without having to sort through the entire prediction structure.  

*Note: True data values from the delphi database can be retrieved using the data_to_df function in evaluation.py. 

In [ ]:
EN.pred_to_array(preds,'New asylum seeking applicants')

The evaluation module can also output a pandas dataframe with the mean of the predictions along with a specified confidence interval for a given indicator variable. There are also options for presenting the true values, residuals, and error bounds based off of the residuals. 

*Note: Setting true_vals = True assumes that real data values exist in the database that match the time points of the predictions. Since the data retrieval function is set to return heuristic estimates for missing data values, then it's possible to have completely "made-up" true data if none actually exist for the prediction time range. Also whatever the mean_pred_to_df function should be passed the same country, state, units arguments as train_model (if any were passed). 

In [ ]:
EN.mean_pred_to_df(preds,'New asylum seeking applicants',ci=None,true_vals=True,use_heuristic_for_true=True)

Here we can see the over all Root Mean Squared Error (RMSE) for a specific indicator.

*Note: This uses the true values for a given indicator and thus the above note given for mean_pred_to_df also holds true for here too.

In [ ]:
EN.calculate_prediction_rmse(preds,'New asylum seeking applicants')

Finally we can get a plots representing the same data shown above. 

The plot types are:
- Prediction: Shows only the predictions with specified confidence intervals. This is the default setting.
- Comparison: Shows the predictions and confidence intervals along with a curve representing the true data values.
- Error: Plots just the error with the error bounds along with a red reference line at 0. 

Additionally you may set the confidence interval significance and also request to see the RMSE posted on the plot.

*Note: The above note for mean_pred_to_df also holds true for the Comparison and Error plot type. Also any other string argument passed to plot_type results in the defaults in the 'Prediction' plot type. The save_as argument can be set to a filename (with extension) to save the plot as a file (e.g, save_as = pred_plot.pdf). The parameter show_rmse is ignored for the "Prediction" type plot. 

In [ ]:
EN.pred_plot(preds,'New asylum seeking applicants',0.99,plot_type='Comparison',show_rmse=True, show_training_data=False)
